### 输入为多的情况

In [3]:
from keras.models import Model
from keras import layers
from keras import Input

text_vocabulary_size = 10000
question_vocabulary_size = 10000
answer_vocabulary_size = 500

text_input = Input(shape=(None,), dtype="int32", name="text")

embedded_text = layers.Embedding(64, text_vocabulary_size)(text_input)

encoded_text = layers.LSTM(32)(embedded_text)

question_input = Input(shape=(None,), dtype="int32", name="question")

embedded_question = layers.Embedding(32, question_vocabulary_size)(question_input)

encoded_question = layers.LSTM(16)(embedded_question)
# 可以用add, subtract, concatenate等方法把多个输入层连接起来。
concatenated = layers.concatenate([encoded_text, encoded_question], axis=-1)

answer = layers.Dense(answer_vocabulary_size, activation="softmax")(concatenated)
# Model把一组有对应关系的input和output转变为模型。
model = Model([text_input, question_input], answer)

model.compile(optimizer="rmsprop", loss="categorical_crossentropy", metrics=["acc"])

In [6]:
import numpy as np

num_samples = 1000
max_length = 100

text = np.random.randint(1, text_vocabulary_size, size=(num_samples, max_length))
question = np.random.randint(1, question_vocabulary_size, size=(num_samples, max_length))
answers = np.random.randint(0, 1, size=(num_samples, answer_vocabulary_size))

model.fit([text, question], answers, epochs=10, batch_size=128)
# 当明确了各输入名的时候，可以用字典的方式传入多个输入。与上面等效。
model.fit({"text": text, "question": question}, answers, epochs=10, batch_size=128)


Epoch 1/10
1000/1000 [==============================] - 6s 6ms/step - loss: nan - acc: 0.9320
Epoch 2/10
1000/1000 [==============================] - 3s 3ms/step - loss: nan - acc: nan
Epoch 3/10
1000/1000 [==============================] - 3s 3ms/step - loss: nan - acc: nan
Epoch 4/10
1000/1000 [==============================] - 3s 3ms/step - loss: nan - acc: 1.0000
Epoch 5/10
1000/1000 [==============================] - 3s 3ms/step - loss: nan - acc: nan
Epoch 6/10
1000/1000 [==============================] - 3s 3ms/step - loss: nan - acc: 1.0000
Epoch 7/10
1000/1000 [==============================] - 3s 3ms/step - loss: nan - acc: 1.0000
Epoch 8/10
1000/1000 [==============================] - 3s 3ms/step - loss: nan - acc: nan
Epoch 9/10
1000/1000 [==============================] - 3s 3ms/step - loss: nan - acc: 1.0000
Epoch 10/10
1000/1000 [==============================] - 3s 3ms/step - loss: nan - acc: nan
Epoch 1/10
1000/1000 [==============================] - 3s 3ms/step - los

### 输出为多的情况

In [8]:
from keras import layers
from keras import Input
from keras.models import Model

vocabulary_size = 50000
num_income_groups = 10

posts_input = Input(shape=(None, ), dtype="int32", name="posts")
embedded_posts = layers.Embedding(256, vocabulary_size)(posts_input)
x = layers.Conv1D(128, 5, activation="relu")(embedded_posts)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(256, 5, activation="relu")(x)
x = layers.Conv1D(256, 5, activation="relu")(x)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(256, 5, activation="relu")(x)
x = layers.Conv1D(256, 5, activation="relu")(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(128, activation="relu")(x)
# 给输出层分别命名。
age_prediction = layers.Dense(1, name="age")(x)
income_prediction = layers.Dense(num_income_groups, activation="softmax", name="income")(x)
gender_prediction = layers.Dense(1, activation="sigmoid", name="gender")(x)

model = Model(posts_input, [age_prediction, income_prediction, gender_prediction])

In [9]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
posts (InputLayer)              (None, None)         0                                            
__________________________________________________________________________________________________
embedding_6 (Embedding)         (None, None, 50000)  12800000    posts[0][0]                      
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, None, 128)    32000128    embedding_6[0][0]                
__________________________________________________________________________________________________
max_pooling1d_1 (MaxPooling1D)  (None, None, 128)    0           conv1d_1[0][0]                   
____________________________________________________________________________________________

In [10]:
# 对于不同的output需要不同的loss function，实际最小化的是各个loss function之和。
model.compile(optimizer="rmsprop", loss=["mse", "categorical_crossentropy", "binary_crossentropy"])
# 明确了output名称的情况下，也可以传入output：loss function的字典。
model.compile(optimizer="rmsprop", loss={"age": "mse", "income": "categorical_crossentropy", "gender": "binary_crossentropy"})

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [1]:
# 各个output的loss function可能差别很大，导致模型会优先降低较大的loss。这种情况会要给各个loss分配不同的重要性。
model.compile(optimizer="rmsprop", loss=["mse", "categorical_crossentropy", "binary_crossentropy"], loss_weights=[0.25, 1., 10.])
# 明确了output名称的情况下，也可以传入output: loss的字典。
model.compile(optimizer="rmsprop", loss={"age": "mse", "income": "categorical_crossentropy", "gender": "binary_crossentropy"}, / 
              loss_weights={"age": 0.25, "income": 1, "gender": 10})

NameError: name 'model' is not defined

In [ ]:
model.fit(posts, [age_targets, icnome_targets, gender_targets], epochs=10, batch_size=64)

model.fit(posts, {"age": age_targets, "icnome": income_targets, "gender": gender_targets}, epochs=10, batch_size=64)